In [1]:
using LinearAlgebra
using Statistics




In [2]:
function pdb2matrix(pdb_file::String)
    #pdb_txt = readdlm(pdb_file)
    x,y,z = [], [], []
    for line in eachline(open(pdb_file))
        if startswith(line, "ATOM") && strip(line[13:16]) == "CA"
            push!(x, parse(Float64,strip(line[31:38])))
            push!(y, parse(Float64,strip(line[39:46])))
            push!(z, parse(Float64,strip(line[47:54])))
        end
    end
    m = hcat(x,y,z)
    return m
end

function xyz2matrix(input_xyz)
    # Imports .xyz file format as data frame, and converts it to an N x 3 Matrix
    raw_xyz=readdlm(input_xyz)
    natoms=size(raw_xyz)[1]
    just_coords=Array{Float64}(raw_xyz[2:natoms,2:4])
    return just_coords
end

function translate_to_centroid(coord_matrix)
    # Normalises the molecular coordinates by centering them.
    center = [mean(coord_matrix[:,1]);mean(coord_matrix[:,2]);mean(coord_matrix[:,3])]
    centroid = transpose(center)
    translated_geom = broadcast(-,coord_matrix,centroid)
    return translated_geom
end

function cross_covariance_matrix(Pmatrix,Qmatrix)
    # Cross covariance matrix gives measure of variability between two matrices
    CCmatrix = transpose(Pmatrix) * Qmatrix
    return CCmatrix
end

function optimal_rotation_matrix(CCmatrix)
    # Returns 3x3 matrix that can be applied to P to get Q
    ORmatrix = sqrt(transpose(CCmatrix)*CCmatrix)*inv(CCmatrix)
    return ORmatrix
end

function kabsch(m1,m2)

    normalisedP = (translate_to_centroid(m1))
    normalisedQ = (translate_to_centroid(m2))

    xcov = cross_covariance_matrix(normalisedP,normalisedQ)
    orot = optimal_rotation_matrix(xcov)

    num_atoms = size(normalisedP)[1]
    rotated = zeros(Float64,num_atoms,3)

    for i=1:num_atoms
        rotated[i,:] = orot*normalisedP[i,:]
    end

    RMSD_value = norm(rotated-normalisedQ)
    return RMSD_value
end

kabsch (generic function with 1 method)

In [ ]:
function slicematrix(matrix, wsize=5)
    msize = size(matrix)[1]-wsize
    megam = [matrix[i:i+wsize,:] for i in 1:msize]
    return megam
end

slicematrix (generic function with 2 methods)

In [6]:
megax = slicematrix(pdb2matrix("small.pdb"))


UndefVarError: UndefVarError: `m` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [85]:
function kabsh_matrix(megax)    
    n = size(megax)[1]  # Change this to the desired size
    matrix = zeros(Float64, n, n)

    for i = 1:n # Fill the upper triangle
        for j = i:n  # Ensure j >= i for the upper triangle
            matrix[i, j] = kabsch(megax[i], megax[j])
        end
    end
    return matrix
end

kabsh_matrix (generic function with 1 method)

In [86]:
kabsh_matrix(megax)

45×45 Matrix{Float64}:
 2.92342e-14  6.33127      8.84129      …  6.97407      6.88663
 0.0          9.33463e-15  6.53931         5.96541      5.9111
 0.0          0.0          5.87035e-15     5.32664      5.36248
 0.0          0.0          0.0             0.422552     0.373994
 0.0          0.0          0.0             0.495378     0.434438
 0.0          0.0          0.0          …  0.600804     0.591554
 0.0          0.0          0.0             0.347801     0.399047
 0.0          0.0          0.0             0.211718     0.234287
 0.0          0.0          0.0             0.260834     0.231935
 0.0          0.0          0.0             0.25861      0.246963
 ⋮                                      ⋱               
 0.0          0.0          0.0             5.34432      5.32596
 0.0          0.0          0.0             6.91752      6.85814
 0.0          0.0          0.0             7.65428      7.56699
 0.0          0.0          0.0             6.09165      6.02374
 0.0          0.0 

In [ ]:
using PlotlyJS

plot(1:100,2:200)